In [0]:
%run "../includes/configuration"

In [0]:
from pyspark.sql.functions import sum, desc, when, count,col,rank
from pyspark.sql.window import Window

In [0]:
race_result_df = spark.read.parquet(f"{presentation_folder_path}/races_result")

In [0]:
constructor_standing_df = race_result_df\
                    .groupBy('race_year','team')\
                    .agg(sum('points').alias('points'),
                         count(when(col('position') == 1, True)).alias('wins'))\
                    .orderBy(desc('wins'))

In [0]:
constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('points'), desc('wins'))
final_constructor_standing_df = constructor_standing_df.withColumn('constructor_rank', rank().over(constructor_rank_spec))

In [0]:
display(final_constructor_standing_df)

In [0]:
#constructor_standing_df.write.mode('overwrite').parquet(f"{presentation_folder_path}/constructor_standing")

In [0]:
constructor_standing_df.write.mode('overwrite').format('parquet').saveAsTable("f1_presentation.constructor_standing")

In [0]:
%sql
SELECT * FROM f1_presentation.constructor_standing